### 5. Setup

In [1]:
%autoawait
import time
import asyncio
from termcolor import colored,cprint

from aries_basic_controller.aries_controller import AriesAgentController

WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 7982
WEBHOOK_BASE = ""
ADMIN_URL = "http://the_wallet-agent:7981"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL)


IPython autoawait is `on`, and set to use `asyncio`


In [2]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)


### 6. Check existing credential

In [3]:
credential_id = "Operation n"
credential = await agent_controller.credentials.get_by_id(credential_id)
print(credential)

{'referent': 'Operation n', 'attrs': {'speed': '1', 'volt': '1', 'dose': '1', 'type': 'drug_A'}, 'schema_id': 'PEcmuSQ7G3z771J7dse1R5:2:Operation:0.0.5', 'cred_def_id': 'PEcmuSQ7G3z771J7dse1R5:3:CL:87:default', 'rev_reg_id': None, 'cred_rev_id': None}


### 7. Check connection

In [4]:
response = await agent_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 1:
    connection = response['results'][1]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
else:
    print("You must create a connection")
    

Results :  [{'state': 'active', 'invitation_key': '5sUS3jiuXTW96bs68X4iBzPeHGSrymakUFtTfCKFSafQ', 'accept': 'manual', 'updated_at': '2023-01-06 04:54:44.861411Z', 'my_did': '4gqtJKna9qZYztcAGkUKDr', 'their_label': 'HOSPITAL', 'rfc23_state': 'completed', 'their_did': 'UFHBQiVeARLVd576EuePnN', 'connection_id': 'da28eb79-0431-4b7f-9af9-92ed004178a5', 'their_role': 'invitee', 'routing_state': 'none', 'created_at': '2023-01-06 04:54:24.454817Z', 'invitation_mode': 'once'}, {'state': 'active', 'invitation_key': 'E1iouZmg8RHSSxEpqpVBEERCmKwPUHLw6KK6yEiFfCy9', 'accept': 'manual', 'updated_at': '2023-01-06 04:55:22.022784Z', 'my_did': 'GjzGVV4asii6Z3RGDMeNnr', 'their_label': 'MEDICAL_DEVICE_COMPUTER', 'rfc23_state': 'completed', 'their_did': 'NkM1rD63bQng6NMgtomvFQ', 'connection_id': '7b32bc63-733c-4355-bd34-84754a82a9fe', 'their_role': 'invitee', 'routing_state': 'none', 'created_at': '2023-01-06 04:55:01.645190Z', 'invitation_mode': 'once'}]
Connection : {'state': 'active', 'invitation_key': 

### 8. Go to Medical_device_computer notebook to send verify request

### 9. Send verify response

In [5]:
response = await agent_controller.proofs.get_records()
print(response)

print('\n')

state = response['results'][0]["state"]
presentation_exchange_id = response['results'][0]['presentation_exchange_id']
presentation_request = response['results'][0]['presentation_request']

print('Presentation Exchange ID\n')
print(response['results'][0]['presentation_exchange_id'])
print('Presentation Request Object\n')
print(response['results'][0]['presentation_request'])
print('Requested Attributes\n')
print(response['results'][0]['presentation_request']['requested_attributes'])
requested_attribs = response['results'][0]['presentation_request']['requested_attributes']
print('Requested Predicates\n')
print(response['results'][0]['presentation_request']['requested_predicates'])
requested_predicates = response['results'][0]['presentation_request']['requested_predicates']

{'results': [{'presentation_request': {'name': 'Proof of Completion of PyDentity SSI Tutorial', 'version': '1.0', 'requested_attributes': {'0_type_uuid': {'name': 'type', 'restrictions': [{'issuer_did': 'PEcmuSQ7G3z771J7dse1R5'}]}, '0_dose_uuid': {'name': 'dose', 'restrictions': [{'issuer_did': 'PEcmuSQ7G3z771J7dse1R5'}]}, '0_speed_uuid': {'name': 'speed', 'restrictions': [{'issuer_did': 'PEcmuSQ7G3z771J7dse1R5'}]}, '0_volt_uuid': {'name': 'volt', 'restrictions': [{'issuer_did': 'PEcmuSQ7G3z771J7dse1R5'}]}}, 'requested_predicates': {}, 'nonce': '897404293099002789488009'}, 'state': 'request_received', 'role': 'prover', 'thread_id': '33992292-05e7-4dd3-8005-38fb5b3414c3', 'updated_at': '2023-01-06 05:14:46.557055Z', 'initiator': 'external', 'presentation_exchange_id': '7d900d91-7f76-416f-9d4e-665b82cb4872', 'connection_id': '7b32bc63-733c-4355-bd34-84754a82a9fe', 'created_at': '2023-01-06 05:14:46.557055Z', 'presentation_request_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/pres

### 10. Find credential

In [6]:
if state == "request_received":
    print(
    "Received Request -> Query for credentials in the wallet that satisfy the proof request")

# include self-attested attributes (not included in credentials)
credentials_by_reft = {}
revealed = {}
self_attested = {}
predicates = {}

# select credentials to provide for the proof
credentials = await agent_controller.proofs.get_presentation_credentials(presentation_exchange_id)
print(credentials)

if credentials:
    for row in sorted(
        credentials,
        key=lambda c: dict(c["cred_info"]["attrs"]),
        reverse=True,
    ):
        for referent in row["presentation_referents"]:
            if referent not in credentials_by_reft:
                credentials_by_reft[referent] = row

for referent in presentation_request["requested_attributes"]:
    if referent in credentials_by_reft:
        revealed[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"][
                "referent"
            ],
            "revealed": True,
        }
    else:
        self_attested[referent] = "South Africa"

for referent in presentation_request["requested_predicates"]:
    if referent in credentials_by_reft:
        predicates[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"][
                "referent"
            ]
        }

print("\nGenerate the proof")
proof = {
    "requested_predicates": predicates,
    "requested_attributes": revealed,
    "self_attested_attributes": self_attested,
}
print(proof)
print("\nXXX")
print(predicates)
print(revealed)
print(self_attested)

Received Request -> Query for credentials in the wallet that satisfy the proof request
[{'cred_info': {'referent': 'Operation n', 'attrs': {'volt': '1', 'dose': '1', 'type': 'drug_A', 'speed': '1'}, 'schema_id': 'PEcmuSQ7G3z771J7dse1R5:2:Operation:0.0.5', 'cred_def_id': 'PEcmuSQ7G3z771J7dse1R5:3:CL:87:default', 'rev_reg_id': None, 'cred_rev_id': None}, 'interval': None, 'presentation_referents': ['0_speed_uuid', '0_volt_uuid', '0_dose_uuid', '0_type_uuid']}]

Generate the proof
{'requested_predicates': {}, 'requested_attributes': {'0_type_uuid': {'cred_id': 'Operation n', 'revealed': True}, '0_dose_uuid': {'cred_id': 'Operation n', 'revealed': True}, '0_speed_uuid': {'cred_id': 'Operation n', 'revealed': True}, '0_volt_uuid': {'cred_id': 'Operation n', 'revealed': True}}, 'self_attested_attributes': {}}

XXX
{}
{'0_type_uuid': {'cred_id': 'Operation n', 'revealed': True}, '0_dose_uuid': {'cred_id': 'Operation n', 'revealed': True}, '0_speed_uuid': {'cred_id': 'Operation n', 'revealed':

### 11. Send proof back

In [7]:
response = await agent_controller.proofs.send_presentation(presentation_exchange_id, proof)
print(response)

{'presentation_request': {'name': 'Proof of Completion of PyDentity SSI Tutorial', 'version': '1.0', 'requested_attributes': {'0_type_uuid': {'name': 'type', 'restrictions': [{'issuer_did': 'PEcmuSQ7G3z771J7dse1R5'}]}, '0_dose_uuid': {'name': 'dose', 'restrictions': [{'issuer_did': 'PEcmuSQ7G3z771J7dse1R5'}]}, '0_speed_uuid': {'name': 'speed', 'restrictions': [{'issuer_did': 'PEcmuSQ7G3z771J7dse1R5'}]}, '0_volt_uuid': {'name': 'volt', 'restrictions': [{'issuer_did': 'PEcmuSQ7G3z771J7dse1R5'}]}}, 'requested_predicates': {}, 'nonce': '897404293099002789488009'}, 'state': 'presentation_sent', 'role': 'prover', 'thread_id': '33992292-05e7-4dd3-8005-38fb5b3414c3', 'updated_at': '2023-01-06 05:14:57.635126Z', 'initiator': 'external', 'presentation_exchange_id': '7d900d91-7f76-416f-9d4e-665b82cb4872', 'connection_id': '7b32bc63-733c-4355-bd34-84754a82a9fe', 'presentation': {'proof': {'proofs': [{'primary_proof': {'eq_proof': {'revealed_attrs': {'dose': '1', 'speed': '1', 'type': '203295886596

### **Terminate**

In [12]:
response = await agent_controller.terminate()
print(response)

None
